# Calculate the Year to Date Price Trends for 10-year Treasury Notes

## Background
Treasury notes, also called T-notes and treasury bonds, provide a fixed interest rate throughout their lifespan and pay semi-annual coupon payments until they reach maturity. When the note (or bond) matures, the holder receives the face (or par) value of the note (or bond). This sets them apart from treasury bills, as treasury notes (and bonds) offer coupon payments every six months, along with earning interest, for the duration of the security. Treasury notes are offered for 2, 3, 5, 7, and 10 years, whereas bonds mature after 20 or 30 years. They can be priced below or above their face (or par) value, but the lenders are entitled to the full par value of the note upon maturity.

<div>
    <center>
        <img src="figs/Fig-Bond-Asset-Timeline-Schematic.png" width="800"/>
    </center>
</div>


### Pricing
Let the term of a note be T-years with $\lambda$ coupon payments per year; $N = \lambda{T}$ coupon payments over the bond term. Further, let $\bar{c}$ denote the annualized coupon rate, and $\bar{r}$ denote the annual effective market rate of interest. Then, the _fair price_ for note $V_{B}$ is the present value of coupon payments $C$ plus the discounted face (par) value $V_{P}$ of the bond:

$$
V_{B} = \mathcal{D}^{-1}_{N}V_{P}+\sum_{j=1}^{N}\mathcal{D}_{j}^{-1}C
$$

where $\mathcal{D}_{i}$ denotes the discount factor for the period $0\rightarrow{I}$, which can be either a discrete or continuous compounding model. The coupon payment $C=\left(\bar{c}/\lambda\right)\cdot{V_{P}}$ is set when the bond is purchased, and the interest rate $i=\bar{r}/\lambda$ varies with the market (set at auction). The contract between the U.S. government, the issuer, and the note holder (you) includes the note’s coupon rate, maturity date, and face (par) value.

## Learning objectives
Using historical `TMUBMUSD10Y` quotation data, compute the price trend for a 10-year T-Note with a coupon rate of 3.375%. We'll load the historical dataset, which contains daily Year-to-Date (YTD) Open, High, Low, and Close yield values from `01/03/23` to `06/01/23` for 10-year T-notes. You'll compute the price trend from this historical data.

* __Objective 1__: Compute the T-Note price series for historical daily yield values. Store the computed T-Note price values in an array. Each row of the array holds data for a particular date, where each column corresponds to computed price values calculated for the yield at the close of the market using discrete and continuous compounding. 

## Setup
In the following code blocks, we set up the computational environment by including the `Include.jl` file. The `Include.jl` file loads external packages, functions that we will use in this exercise, and custom types to model the components of our problem.

### Packages
`Include.jl` loads several external packages that we will use for our excercise:
* [DataFrames.jl](https://dataframes.juliadata.org/stable/) provides a set of tools for working with tabular data in [Julia](https://julialang.org). Its design and functionality are similar to those of [Pandas (in Python)](https://pandas.pydata.org) and [data.frame, data.table and dplyr (in R)](https://dplyr.tidyverse.org), making it a great general purpose data science tool.
* [Plots.jl](https://docs.juliaplots.org/stable/) is a plotting library in [Julia](https://julialang.org).
* [Optim.jl](https://julianlsolvers.github.io/Optim.jl/stable/) is a [Julia](https://julialang.org) package for performing simple optimization task. We use this package in the Yield to Maturity (YTM) calculations.
* [PrettyTables.jl](https://ronisbr.github.io/PrettyTables.jl/stable/man/html_backend/) is a [Julia](https://julialang.org) package that implements functions to construct and display text-based tables

### Types
`Include.jl` loads some [problem-specific types](https://docs.julialang.org/en/v1/manual/functions/) that will be helpful for the analysis of T-bill pricing. 

* `MyUSTreasuryCouponSecurityModel` is a [mutable type](https://docs.julialang.org/en/v1/manual/types/#Mutable-Composite-Types) holding the par value $V_{P}$, the duration $T$, the market rate of interest rate $\bar{r}$, the coupoun rate $c$ and the number of coupon payments per year $\lambda$ for treasury note and bonds that pay a coupoun. You construct a `MyUSTreasuryCouponSecurityModel` instance using the `build` method described below.
* `DiscreteCompoundingModel` and `ContinuousCompoundingModel` are [immutable types](https://docs.julialang.org/en/v1/manual/types/#Composite-Types) that let our code know which compounding model we wish to use.

### Functions
`Include.jl` loads the following [Julia functions](https://docs.julialang.org/en/v1/manual/functions/):

`loaddatafile(path::String) -> DataFrame` 
 > This function takes a [String](https://docs.julialang.org/en/v1/manual/strings/) path arguement that points to a local comma seprated value file that holds bond rate and pricing information. The pricing and rate information is returned to the caller in a [DataFrame](https://dataframes.juliadata.org/stable/).
 
`build(model::Type{MyUSTreasuryCouponSecurityModel}, data::NamedTuple) -> MyUSTreasuryCouponSecurityModel` 
> This function takes information in the `data` [NamedTuple](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple) argument (the par value, etc.) and returns an instance of the `MyUSTreasuryCouponSecurityModel` [mutable type](https://docs.julialang.org/en/v1/manual/types/#Mutable-Composite-Types).

`price(model::MyUSTreasuryCouponSecurityModel, compounding::T) -> MyUSTreasuryCouponSecurityModel where T <: AbstractCompoundingModel` 
> This function takes a `MyUSTreasuryCouponSecurityModel` instance, and a compounding model and returns an updated `MyUSTreasuryCouponSecurityModel` instance that holds the price $V_{B}$ in the price field.

`YTM(model::MyUSTreasuryCouponSecurityModel, compounding::T; rₒ::Float64 = 0.01) where T <: AbstractCompoundingModel` 
> This function computes the Yield to Maturity (YTM) given a `MyUSTreasuryCouponSecurityModel` instance, a compounding model and an initial guess for the interest rate (default value of 1\%)

In [1]:
include("Include.jl");

## Objective 1: Compute the T-note price series for historical daily yield values

### Load historical data set
The historical data is stored in a local comma-separated value file named `TMUBMUSD10Y-d-01-03-23-to-06-01-23.csv`. We create the path to the data file and store this value in the `path_to_data_file` variable. We then pass the path to the `loaddatafile(...)` function. The bond data is stored as a [DataFrame](https://dataframes.juliadata.org/stable/) in the `dataset` variable:

In [13]:
path_to_data_file = joinpath(_PATH_TO_DATA,"TMUBMUSD10Y-d-01-03-23-to-06-01-23.csv");
dataset = loaddatafile(path_to_data_file);

### Compute T-Note price series using historical dataset
Set the invariant parameters for the note, e.g., the maturity $T$ in years, the coupon rate $c$, the face value of the note $V_{P}$ and the number of coupon payments per year $\lambda$. Then determine how many records we have in the `dataset` using the `nrow(..)` function, and create (and store) an instance of the `DiscreteCompoundingModel` type in the `discrete_compounding` variable:

In [10]:
T,c,Vₚ,λ = 10.0, 0.03375, 100.0, 2
number_of_days = nrow(dataset)
discrete_compounding = DiscreteCompoundingModel();

Finally, create and populate the `notes` array, which holds instances of `MyUSTreasuryCouponSecurityModel` for each note in the data set. Process each record in a `for` loop:

In [15]:
notes = Array{MyUSTreasuryCouponSecurityModel,1}(undef, number_of_days);
for i ∈ 1:number_of_days
   
    # yeild for date i -
    r̄ᵢ = dataset[i,:Close]*(1/100);
        
    # model discrete compounding -
    model = build(MyUSTreasuryCouponSecurityModel, (
        par = Vₚ, T = T, λ = λ, rate = r̄ᵢ, coupon = c
    )) |> discrete_compounding;
    
    # store result -
    notes[i] = model
end

### Construct a 10-year T-note price series table
Display the calculated T-note prices using the [PrettyTables.jl](https://github.com/ronisbr/PrettyTables.jl) package. In particular, store the date, the coupon rate (as a percentage), the effective interest rate (as a percentage),  the face (par) value, the calculated note $\hat{V}_{B}$ price in the `note_table_data_array`. Display the table by calling the `pretty_table(...)` function: 

In [20]:
note_table_data_array = Array{Any,2}(undef, number_of_days, 5);
note_table_header_array = (["Date","Coupon","Rate","Face","Price"],["yyyy-mm-dd","%","%","USD","USD"])
for i ∈ 1:number_of_days
    
    notes_model = notes[i];
    
    
    note_table_data_array[i,1] = dataset[i,:Date];
    note_table_data_array[i,2] = (notes_model.coupon)*100.0
    note_table_data_array[i,3] = (notes_model.rate)*100.0
    note_table_data_array[i,4] = notes_model.par
    note_table_data_array[i,5] = notes_model.price
end

pretty_table(note_table_data_array, header=note_table_header_array, tf=tf = tf_html_default)

Date,Coupon,Rate,Face,Price
yyyy-mm-dd,%,%,USD,USD
2023-06-01,3.375,3.6,100.0,98.1245
2023-05-31,3.375,3.647,100.0,97.7379
2023-05-30,3.375,3.692,100.0,97.3694
2023-05-26,3.375,3.806,100.0,96.4431
2023-05-25,3.375,3.822,100.0,96.3139
2023-05-24,3.375,3.75,100.0,96.8968
2023-05-23,3.375,3.701,100.0,97.2959
2023-05-22,3.375,3.715,100.0,97.1816
2023-05-19,3.375,3.683,100.0,97.4429
